<a href="https://colab.research.google.com/github/andohyung/MS_Project/blob/main/BART_%EC%9A%94%EC%95%BD%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BART 사용 요약 모델


In [ ]:
# 필요한 라이브러리 설치
!pip install transformers

In [11]:
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

# BART 요약 모델과 토크나이저 로드
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# 요약할 파일 경로 설정
file_path = "/content/drive/MyDrive/멋사_실전프로젝트1/프로젝트1_입력_논문리스트/bert논문/수정1_bert논문.txt"

# 텍스트 파일 읽기
try:
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
except FileNotFoundError:
    print("파일을 찾을 수 없습니다. 경로를 확인해 주세요.")
    exit()

# 토크나이징 및 입력 텐서로 변환
inputs = tokenizer(text, max_length=1024, truncation=True, return_tensors="pt")

# 요약 생성
summary_ids = model.generate(inputs["input_ids"], max_length=250, min_length=100, length_penalty=1.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Summary:", summary)


Summary: BERT is designed to pre-train deep bidirectional representations fromunlabeled text by jointly conditioning on both left and right context in all layers. It obtains new state-of-the-art re-sults on eleven natural language processingtasks, including pushing the GLUE score to80.5% (7.7% point absolute improvement),MultiNLI accuracy to 86.7%, and SQuAD v1.1 question answer-ing Test F1 to 93.2 (1.5 point absolute im-provement)


ROUGE 평가

In [12]:
pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=065eeb3f58af33ea6e010d930423d50236752c993dde74a02366a953f1fdacaa
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [19]:
from transformers import BartTokenizer, BartForConditionalGeneration
from rouge_score import rouge_scorer
import torch

# BART 요약 모델과 토크나이저 로드
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# 요약할 파일 경로 설정
file_path = "/content/drive/MyDrive/멋사_실전프로젝트1/프로젝트1_입력_논문리스트/bert논문/수정1_bert논문.txt"  # 파일 경로 설정

# 실제 요약 텍스트 (평가할 요약의 참조용 텍스트 입력)
reference_summary = "BERT (Bidirectional Encoder Representations from Transformers) is a language model that learns deep bidirectional representations by considering both left and right context. It uses Masked Language Modeling and Next Sentence Prediction to improve understanding across tasks. BERT achieves state-of-the-art results on multiple NLP benchmarks with minimal task-specific adjustments, outperforming prior unidirectional models like GPT. Its open-source release has made it a cornerstone in NLP research."

# 텍스트 파일 읽기
try:
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
except FileNotFoundError:
    print("파일을 찾을 수 없습니다. 경로를 확인해 주세요.")
    exit()

# 토크나이징 및 입력 텐서로 변환
inputs = tokenizer(text, max_length=1024, truncation=True, return_tensors="pt")

# 요약 생성
summary_ids = model.generate(
    inputs["input_ids"],
    max_length=150,      # 최대 요약 길이 설정
    min_length=50,      # 최소 요약 길이 설정
    length_penalty=2.0,  # 길이 페널티
    num_beams=4,         # 빔 서치
    early_stopping=True
)
generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# ROUGE 점수 계산
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_summary, generated_summary)

# 결과 출력
print("Generated Summary:", generated_summary)
print("\nROUGE Scores:")
print("ROUGE-1:", scores['rouge1'])
print("ROUGE-2:", scores['rouge2'])
print("ROUGE-L:", scores['rougeL'])


Generated Summary: We introduce a new language representa-tion model called BERT, which stands forBidirectional Encoder Representations fromTransformers. BERT is designed to pre-train deep bidirectional representations fromunlabeled text by jointly conditioning on both left and right context in all layers. It obtains new state-of-the-art re-sults on eleven natural language processing tasks.

ROUGE Scores:
ROUGE-1: Score(precision=0.5, recall=0.38571428571428573, fmeasure=0.435483870967742)
ROUGE-2: Score(precision=0.18867924528301888, recall=0.14492753623188406, fmeasure=0.16393442622950818)
ROUGE-L: Score(precision=0.37037037037037035, recall=0.2857142857142857, fmeasure=0.3225806451612903)


Fine-tuning 진행할 데이터